---

# CSCI 3202, Fall 2022
# Final Project
# Project Due: Thursday December 8, 2022 at 6:00 PM
## Proposals Due: Friday November 18, 2022 at 6:00 PM


You have two options for completing your final project for this course. 

#### Option 1 ####
The first option is presented in this notebook and involves implementing a Connect Four game with AB pruning and A* as player strategies. 

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 


**By writing your name below, you agree to abide by these rules:**

**Your name:** Giuliano Costa


---

Some useful packages and libraries:



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

### (1a)   Defining the Connect Four class structure

We've provided the humble beginnings of a Connect Four game. You need to fill in this class structure for Connect Four using what we did during class as a guide, and then implement min-max search with AB pruning, and A* search with at least one heuristic function. The class structure includes the following:

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [33]:
class State:
    def __init__(self, moves):
        self.to_move = 'R'
        self.utility = 0
        self.board = {}
        self.moves = moves

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.moves = moves
        self.state = State(moves)

    def result(self, move, state): #NOT HYPOTHETICAl, simply computes a new state...
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        #print("\n" , self.state.to_move, " Turn")
        new_state = cp.deepcopy(state)
        
        #get move column:
        col_move = move[1]
        row_move = 1
        #print("row: ", row_move, "")
        #print("move column: ", col_move, "\n")
        #print("Result of , ", (1, col_move))
        
        #Increment rows and find bottom 
        new_row_move = row_move
        #if move[0] < self.nrow: #if row number is 
        for i in range(1, self.nrow + 1):
            if ((i, col_move) in self.state.moves):
               # print(i, " row not occupied")
                new_row_move = i
        
        #Set new disc dropped
        move = (new_row_move, col_move)
        
        if(move in self.state.moves):
            #print("move is available!")
            new_state.board[move] = state.to_move
            #print("Occupy grid slot: ", move)
           # self.display(display_board = new_state.board
            new_state.moves.remove(move)
            new_state.utility = self.compute_utility(move, state)
        
        #else:
         #   print("move not available!")
            
        
        if state.to_move == 'R':
            new_state.to_move = 'B'
        if state.to_move == 'B':
            new_state.to_move = 'R'            

        return new_state

    def compute_utility(self, move, state): #Computes the move
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = state.to_move

        row, col = move
        player = state.to_move
        #print("Player to move: ", player)
        ncol = self.nrow
        nrow = self.ncol
        nwin = self.nwin
        
        # what are all the ways 'player' could with with 'move'?

        # check for row-wise win
#        in_a_row = 0
#        for c in range(1, ncol + 1):
#           in_a_row += board.get(( row,c )) == player

    # Check horizontal locations for win
        in_a_row = 0
        for c in range(ncol - 3):
            for r in range(nrow):
                if board.get((r, c)) == player and board.get((r, c + 1)) == player and board.get((r, c + 2)) == player and board.get((r, c + 3)) == player:
                    in_a_row = 4
    

        # check for column-wise win
        in_a_col = 0
        for c in range(ncol):
            for r in range(nrow - 3):
                if board.get((r, c)) == player and board.get((r + 1, c)) == player and board.get((r + 2, c)) == player and board.get((r + 3, c)) == player:
                    in_a_col = 4


        # check for NW->SE diagonal win
        # Check positively sloped diags
        in_a_diag1 = 0
        for c in range(ncol - 3):
            for r in range(nrow - 3):
                if board.get((r, c)) == player and board.get((r + 1, c + 1)) == player and board.get((r + 2, c + 2)) == player and board.get((r + 3, c + 3)) == player:
                    in_a_diag1 = 4
     
        #negatively sloped
        in_a_diag2 = 0
        for c in range(ncol - 3):
            for r in range(nrow - 3):
                if board.get((r, c)) == player and board.get((r - 1, c + 1)) == player and board.get((r - 2, c + 2)) == player and board.get((r - 3, c + 3)) == player:
                    in_a_diag2 = 4    
            
        
        #If 4 in a row, then return the winner.
        if nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            print ("Win by:" , [in_a_row, in_a_col, in_a_diag1, in_a_diag2])
            return 1 if player == 'R' else -1
        else:
            return 0 

    def game_over(self, state):
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player): #Renamed to winner, why the fuck was this called utility()
        # "returns the value of the current utility, 1 if red, -1 if black"
        return state.utility if player == 'R' else -state.utility

    def display(self, display_board):
        board = display_board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of Connect Four!'''
        #print(self.state)
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn <= turn_limit:
            for player in [player1, player2]: #alternate players
                turn += 1 
                move = player(self)
                self.state = self.result(move, self.state)
                #self.display(display_board = self.state.board)
                if self.game_over(self.state):
                    self.display(display_board = self.state.board)
                    if self.state.utility == 1:
                        print("Red Wins!")
                    if self.state.utility == -1:
                        print("Black Wins!")     
                    if self.state.utility == 0:
                        print("Tie!")                                             
                    return self.state.utility         
        
                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [34]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''

    possible_actions = game.moves
    random_move = possible_actions[np.random.randint(low=0, high=len(possible_actions))]

    return random_move


In [46]:
test = ConnectFour(nrow=6, ncol=7)

test.play_game(random_player, random_player)
#test.display(test.state.board)

We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of ConnectFour against one another, whoever goes first will win about 55% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.**(Chris: is this true for TicTacToe, or Connect Four)**

**"Unit tests":** If you are wondering how close is close enough to 55%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 52-59%.

In [49]:
# 1000 games between two random players

# Your code here
rwins = 0
bwins = 0
ties = 0
for i in range (0, 1000):
    test = ConnectFour()
    res = test.play_game(random_player, random_player)
    if res == -1:
        bwins += 1
    elif res == 1:
        rwins += 1
    elif res == 0:
        ties += 1
        
print(rwins/(1000-ties), " ", bwins/(1000-ties))


Win by: [0, 4, 0, 0]
. . . . B . . 
. R . . R . . 
. R . . B B B 
. R . . B B R 
. R . B R B B 
R B R R R R B 
Win by: [4, 0, 0, 0]
. B . . . . B 
. B . . R . R 
R R B . B . B 
B R R R R B B 
R R B R B B R 
R B B R B R R 
Win by: [0, 4, 0, 0]
B . . . . . . 
R . R R . . . 
B . B R . . . 
B . B R . B . 
R . R R B R . 
B R B B R B R 
Win by: [4, 0, 0, 0]
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
B . . . . . . 
R R R R B B . 
Win by: [4, 0, 0, 0]
R . . . . . . 
B . . . . . . 
R . R . . . R 
R . B . . . B 
B B B B . . R 
B R R B . . R 
Win by: [4, 0, 0, 0]
. . . . . . . 
. . . . . . . 
. . . . . . . 
. B . . . R . 
. R R R R B R 
B B R R B B B 
Win by: [4, 0, 0, 0]
. . . . . . . 
. . . . . . . 
. . . . . . . 
. B B . . R . 
B R R R R B . 
B R B R R B . 
Win by: [4, 0, 0, 0]
B . . . R . . 
R . . R B B R 
R . . B B R B 
R R R R R R B 
B R B B B B R 
B B R R R R B 
Win by: [0, 4, 0, 0]
. . R . . R B 
. B B B R B B 
B R B R R B R 
R B B R R B R 
B R R B R B R 
R B R R B B B 
W

### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [9]:
# 1000 games between two random players

# Your code here

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Connect Four game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [10]:
# Your code here

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 6x7 ConnectFour board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [11]:
# Your code here

### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [12]:
# Your code here.

### (2) A* Search

In Part II of this project, you need to implement a player strategy to employ A* Search in order to win at ConnectFour. To test your A* player, play 10 games against the random player and 10 games against the AB pruning player. 

Write an explanation of this strategy's implementation and performance in comparison to the random player and the AB Pruning player from **1d**.

A lot of the code that you wrote for the minimax player and the Connect Four game structure can be reused for the A* player. However, you will need to write a new utility function for A* that considers the path cost and heuristic cost.


### (2a) Define a heuristic function
Your A* player will need to use a heuristic function. You have two options for heurstics: research an existing heuristic for Connect Four, or games similar to Connect Four, and use that. Or, design your own heuristic. Write a one-paragraph description of the heuristic you're using, including a citation if you used an existing heuristic.

### (2b) Compare A* to other algorithms
Next, play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player. In four or five paragraphs, report on the outcome. Did one player win more than the other? How often was the game a draw? How many moves did each player make? Were there situations where one player appeared to do better than the other? Given the outcome, are there other heuristics you would like to implement?

In [13]:
# Your code here.